In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/raroog/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Example 3

$\frac{d^2}{dx^2}\Psi+\frac{1}{5}\frac{d}{dx}\Psi+\Psi=-\frac{1}{5}\exp(-\frac{x}{5})\cos(x)$

With boundary initial condition $\Psi(0)=0$, $\frac{d}{dx}\Psi(0)=1$ and domain $x\in[0,2]$

In [2]:
X_train = np.arange(0, 2., 0.2) + 1e-8
X_train = X_train.reshape(-1,1)
X_test = np.arange(0, 2., 0.01) + 1e-8
X_test = X_test.reshape(-1,1) 

In [3]:
inits = [{'variable':0, 'value':0, 'type':'dirichlet',
        'function':lambda X: tf.constant(0., dtype='float64', shape=(X.shape[0],1))},
        {'variable':0, 'value':0, 'type':'neumann',
        'function':lambda X: tf.constant(1., dtype='float64', shape=(X.shape[0],1))}]

In [4]:
class TrialSolution(tf.keras.models.Model):
  def __init__(self, conditions, n_i, n_h, n_o=1, activation='sigmoid', equation_type='ODE'):
    super(TrialSolution, self).__init__()
    
    # Dimensions of the network
    self.n_i = n_i
    self.n_h = n_h
    self.n_o = n_o
    
    # Boundary conditions
    self.conditions = conditions
    
    # Shallow network
    self.hidden_layer = tf.keras.layers.Dense(units=self.n_h, activation=activation)
    self.output_layer = tf.keras.layers.Dense(units=self.n_o, activation='linear')
    
  def call(self, X):
    X = tf.convert_to_tensor(X)
    response = self.hidden_layer(X)
    response = self.output_layer(response)
    
    # Automatic conditions incorporation including Neumann BCs
    # It should be used to generate the *call* method instead of calculating it every damned time
        
    boundary_value = tf.constant(0., dtype='float64', shape=response.get_shape())
    
    for condition in self.conditions:
      vanishing = tf.constant(1., dtype='float64', shape=response.get_shape())
      temp_bc = 0
      if condition['type'] == 'dirichlet':
        temp_bc = tf.reshape(condition['function'](X), shape=boundary_value.shape)           
        for vanisher in self.conditions:
          if vanisher['variable'] != condition['variable'] and vanisher['value'] != condition['value']:
            if vanisher['type'] == 'dirichlet':
              vanishing *= (X[:, vanisher['variable']]
                                        - tf.constant(vanisher['value'], dtype='float64', shape=boundary_value.shape))
            elif vanisher['type'] == 'neumann':
              vanishing *= (X[:, vanisher['variable']]
                                        - tf.constant(vanisher['value'], dtype='float64', shape=boundary_value.shape))
        boundary_value += temp_bc * vanishing
        response *= (tf.constant(condition['value'], dtype='float64', shape=boundary_value.shape)
                     - tf.reshape(X[:, condition['variable']], shape=boundary_value.shape))
      elif condition['type'] == 'neumann':
        temp_bc = (tf.reshape(condition['function'](X), shape=boundary_value.shape)
                   * tf.reshape(X[:, condition['variable']], shape=boundary_value.shape))
        boundary_value = temp_bc
        response *= (tf.constant(condition['value'], dtype='float64', shape=boundary_value.shape)
                     - tf.reshape(X[:, condition['variable']], shape=boundary_value.shape))  
    response += boundary_value
    return response

The trial solution for this case is $\Psi(x)=x + x^2N(x)$.
The first function below is the function $A(x)=x$
and the second function is the function $B(x)=x^2$.

In [5]:
ts = TrialSolution(conditions=inits, n_i=1, n_h=10, n_o=1)

### Defining the loss function for a single point and a whole set

The loss function is based on the formula:
$$Loss(N)=\sum_i \left(L\Psi(x_i, N(x_i))-f(x_i,\Psi(x_i, N(x_i))) \right)^2$$
Where $N(x)$ is the neural network and $L$ is some differential operator.

In [6]:
def diff_loss(network, inputs):
  # Compute the gradients
  with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape:
      inputs = tf.convert_to_tensor(inputs)
      tape.watch(inputs)
      tape2.watch(inputs)
      response = network(inputs)  
    grads = tape.gradient(response, inputs)
  laplace = tape2.gradient(grads, inputs)
  
  # Compute the loss
  loss = tf.square(laplace + tf.constant(0.2, dtype='float64')*grads + response
          + tf.constant(0.2, dtype='float64')*tf.exp( tf.constant(-0.2, dtype='float64') * inputs)
                   * tf.cos(inputs))
  return loss

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
train_loss = tf.keras.metrics.Mean('train')

In [8]:
@tf.function
def train_step(X):
  # Online learning
  for i in X:
    with tf.GradientTape() as tape:
      loss = diff_loss(ts, tf.reshape(i, shape=(1,1)))
    gradients = tape.gradient(loss, ts.trainable_variables)
    optimizer.apply_gradients(zip(gradients, ts.trainable_variables))
  
  train_loss(diff_loss(ts, X))

Let's check if everything is fine.

In [ ]:
ts(tf.convert_to_tensor(X_train))

<tf.Tensor: id=119, shape=(10, 1), dtype=float64, numpy=
array([[1.00000000e-08],
       [2.04050755e-01],
       [4.15765075e-01],
       [6.34527507e-01],
       [8.59812173e-01],
       [1.09122143e+00],
       [1.32851771e+00],
       [1.57164710e+00],
       [1.82075389e+00],
       [2.07618592e+00]])>

### Training

In [ ]:
EPOCHS = 100000
for epoch in range(EPOCHS):
  train_step(X_train)
  if (epoch+1) % 1000 == 0:
    print(train_loss.result().numpy())

0.0030667903
0.0015638769
0.0010593429
0.00080488593
0.0006507894
0.0005471063
0.0004723987
0.00041592174


### Plotting the results 

The numerical solution (training set - red, valdiaiton set - green) along with the analytical solution (blue).

In [ ]:
pred_train = ts.call(tf.convert_to_tensor(X_train, dtype='float64')).numpy()
pred_test = ts(tf.convert_to_tensor(X_test, dtype='float64')).numpy()
plt.scatter(X_train, pred_train, c='r', label='Numerical - Training', marker='+', s=30)
plt.plot(X_test, pred_test, c='g', label='Numerical - Test')
plt.plot(X_test, np.exp(-0.2*X_test)*np.sin(X_test), c='b', label='Analytic')
plt.legend()
plt.show()

Let's check the errors on the training set.

In [ ]:
plt.plot(X_train, pred_train - np.exp(-0.2*X_train)*np.sin(X_train), label='Error - Train')
plt.legend()
plt.show()

Let's check the errors on the test set.

In [ ]:
plt.plot(X_test, pred_test - np.exp(-0.2*X_test)*np.sin(X_test), label='Error - Test')
plt.legend()
plt.show()

The mean loss calculated on the test set.

In [ ]:
diff_loss(ts, X_test).numpy().mean()

Mean absolute error on the train set.

In [ ]:
np.abs(pred_train - np.exp(-0.2*X_train)*np.sin(X_train)).mean()

Mean absolute error on the test set - interpolation error.

In [ ]:
np.abs(pred_test - np.exp(-0.2*X_test)*np.sin(X_test)).mean()